# Object Detection in Fifa using Tensorflow Object Detection API

## 1 - Configuring environment

In [0]:
# Installing prerequisites
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive
!pip install -q pycocotools

In [0]:
# Cloning tensorflow models repo
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [0]:
# Configuring protoc
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


## 2 - Uploading and Extracting Dataset

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import zipfile

local_zip = '/content/drive/My Drive/Dataset_FIFA.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

## 3 - Generating csv with the xml files

In [0]:
import os
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET

%cd /content/Dataset_FIFA

data_dir = "/content/data"

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir)

def xml_to_csv(path, directory):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = ("/content/Dataset_FIFA/" + directory + '/' + root.find('filename').text, # Filename
                     int(root.find('size')[0].text), # Width
                     int(root.find('size')[1].text), # Height
                     member[0].text, # Class
                     int(member[4][0].text), # Xmin
                     int(member[4][1].text), # Ymin
                     int(member[4][2].text), # Xmax
                     int(member[4][3].text) # Ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    for directory in ['train', 'test']:
        image_path = os.path.join(os.getcwd(), directory)
        xml_df = xml_to_csv(image_path, directory)
        xml_df.to_csv('/content/data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')


main()

/content/Dataset_FIFA
Successfully converted xml to csv.
Successfully converted xml to csv.


## 4 - Configuring paths and installing OD library

In [0]:
# From tensorflow/models/
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [0]:
!python3 /content/models/research/setup.py install

running install
running bdist_egg
running egg_info
creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/model_tpu_main.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection

## 5 - Converting CSV file to TF record

### Upload generation_tfrecord.py file

In [0]:
import shutil

shutil.move("/generate_tfrecord.py", "/content")

'/content/generate_tfrecord.py'

### Converting files

In [0]:
%cd /content
!python3 generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record --image_dir=content/images/train

/content
Successfully created the TFRecords: /content/data/train.record


In [0]:
%cd /content
!python3 generate_tfrecord.py --csv_input=data/test_labels.csv --output_path=data/test.record --image_dir=content/images/test

/content
Successfully created the TFRecords: /content/data/test.record


## 6 - Uploading and Moving files to models/research/object_detection

### Upload ssd_mobilenet_v2_coco_2018_03_29.zip on drive before

In [0]:
import os
import tarfile

local_tar_gz = '/content/drive/My Drive/ssd_mobilenet_v2_coco_2018_03_29.tar.gz'
tar_gz_ref = tarfile.open(local_tar_gz, "r:gz")
tar_gz_ref.extractall('/content/models/research/object_detection')
tar_gz_ref.close()

### Upload training.zip on drive before

In [0]:
import os
import shutil

training_dir = "/content/models/research/object_detection/training"

if os.path.exists(training_dir):
    shutil.rmtree(training_dir)
os.makedirs(training_dir)

local_zip = '/content/drive/My Drive/training.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/models/research/object_detection')
zip_ref.close()

In [0]:
import shutil

final_dir = "/content/models/research/object_detection"

shutil.move("/content/data/test.record", final_dir + "/data")
shutil.move("/content/data/test_labels.csv", final_dir + "/data")
shutil.move("/content/data/train.record", final_dir + "/data")
shutil.move("/content/data/train_labels.csv", final_dir + "/data")
shutil.move("/content/Dataset_FIFA", final_dir)

'/content/models/research/object_detection/Dataset_FIFA'

## 7 - Training

### Remove /content/models/research/object_detection/legacy/train.py file and upload train.py before running the next cell

In [0]:
import shutil

shutil.move("/train.py", "/content/models/research/object_detection/legacy")

'/content/models/research/object_detection/legacy/train.py'

In [0]:
%cd /content/models/research
!python3 object_detection/legacy/train.py --logtostderr --train_dir=object_detection/training/ --pipeline_config_path=object_detection/training/ssd_mobilenet_v2_coco.config

/content/models/research

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use the `axis` argumen

KeyboardInterrupt: ignored

## 8 - Running Tensorboard

In [0]:
import shutil

graph_dir = "/content/models/research/object_detection/training"

shutil.move("/content/models/research/object_detection/training/events.out.tfevents.1550585191.c9020eceeeda", graph_dir)
shutil.move("/content/models/research/object_detection/training/events.out.tfevents.1550596284.c9020eceeeda", graph_dir)

Error: ignored

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-02-19 21:04:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.111.123, 52.72.145.109, 52.54.84.112, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.111.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.91MB/s    in 0.6s    

2019-02-19 21:04:16 (8.91 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = '/content/models/research/object_detection/training'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://272fe33c.ngrok.io


## 9 - Testing the model

### Uploading export_inference_graph.py file before running next cell

In [0]:
import shutil

shutil.move("/content/export_inference_graph.py", "/content/models/research/object_detection")

'/content/models/research/object_detection/export_inference_graph.py'

In [0]:
# Exporting the inference graph

%cd /content/models/research/object_detection

!python3 export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path training/ssd_mobilenet_v2_coco.config \
    --trained_checkpoint_prefix training/model.ckpt-17959 \
    --output_directory fifa_inference_graph

/content/models/research/object_detection
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
155 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                   

### Copy test images in content/models/research/object_detection/images/test to content/models/research/object_detection/test_images

In [0]:
import os
import shutil

initial_dir = "/content/models/research/object_detection/Dataset_FIFA/test/"
final_dir = "/content/models/research/object_detection/test_images/"

name = "fkrl"

for i in range(131,151):
  format_string = name + str(i) + ".png"
  shutil.move(initial_dir + format_string, final_dir + format_string)

In [0]:
import os

%cd /content/models/research/object_detection

# What model to download.
MODEL_NAME = 'fifa_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

NUM_CLASSES = 1

/content/models/research/object_detection


In [0]:
PATH_TO_TEST_IMAGES_DIR = "/content/models/research/object_detection/test_images/"
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'fkrl{}.png'.format(i)) for i in range(131, 151) ]

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
import matplotlib.pyplot as plt

from utils import label_map_util

from utils import visualization_utils as vis_util

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
from google.colab import files

# Size, in inches, of the output images.
IMAGE_SIZE = (30, 22)
i = 1

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  plt.savefig("/content/models/research/object_detection/test_images/image_labeled" + str(i))
  files.download("/content/models/research/object_detection/test_images/image_labeled" + str(i) + ".png")
  i += 1

/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [0]:
## Feature extractor MobileNet v2
import cv2

def run_feature_extractor_for_single_image(image_path=None):
  with detection_graph.as_default():
    with tf.Session() as sess:
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      feature_vector = detection_graph.get_tensor_by_name(
          "FeatureExtractor/MobilenetV2/layer_19_2_Conv2d_5_3x3_s2_128/Relu6:0")

      print(feature_vector)
      
      image = cv2.imread(image_path)
      
      image_np = cv2.resize(image, (900, 400))
      image_np_expanded = np.expand_dims(image_np, axis=0)

      rep = sess.run([feature_vector], feed_dict={image_tensor: image_np_expanded})
      input_window = np.array(rep).reshape(-1, 128)
      
      print(rep)
      print(input_window)

In [0]:
image1 = TEST_IMAGE_PATHS[0]

run_feature_extractor_for_single_image(image1)

Tensor("FeatureExtractor/MobilenetV2/layer_19_2_Conv2d_5_3x3_s2_128/Relu6:0", shape=(?, ?, ?, 128), dtype=float32)
[array([[[[1.4458029e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
          2.5898404e+00, 8.8033450e-01, 1.1792274e+00, 3.8126068e+00,
          3.5060735e+00, 2.7614121e+00, 0.0000000e+00, 1.8403063e+00,
          3.1540523e+00, 3.0859118e+00, 0.0000000e+00, 4.7184286e+00,
          7.8025436e-01, 0.0000000e+00, 0.0000000e+00, 4.6056480e+00,
          2.4768295e+00, 0.0000000e+00, 6.0000000e+00, 0.0000000e+00,
          5.9901047e-01, 4.5754595e+00, 0.0000000e+00, 3.8997884e+00,
          4.3291092e-02, 0.0000000e+00, 5.2072859e+00, 0.0000000e+00,
          0.0000000e+00, 3.8998199e+00, 4.8418045e-03, 3.4036446e-01,
          6.0000000e+00, 4.6418009e+00, 2.0282602e-01, 3.1356983e+00,
          2.8852463e-02, 1.9020758e+00, 1.5011468e+00, 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 4.0655141e+00,
          0.0000000e+00, 4.5037913e+00, 4.50